In [1]:
import pandas as pd
import numpy as np

In [7]:
%%time
import vars

######################### CRIA df_rat
df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')

df_rat = df_rat[
    (df_rat['NAT.CODIGO'].str.contains('Y0700[1345]')) |
    (df_rat['NAT.CODIGO'] == 'Y04012')  
]

df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

######################### BUSCA DADOS DE VIATURAS E ACRESCENTA EM df_rat
df_ratv = pd.read_sql_table('tbl_rat_veiculo', 'sqlite:///gdo.db')
df_ratv = df_ratv['NUM_ATIVIDADE'].value_counts()
df_ratv.rename('VIATURAS', inplace=True)
df_rat = df_rat.join(df_ratv, how='left')
del df_ratv
df_rat['VIATURAS'].fillna(0, inplace=True)
df_rat['VIATURAS'] = df_rat['VIATURAS'].astype(int)

######################### BUSCA DADOS DE EFETIVO E ACRESCENTA EM df_rat
df_rate = pd.read_sql_table('tbl_rat_efetivo', 'sqlite:///gdo.db')
df_rate = df_rate['NUM_ATIVIDADE'].value_counts()
df_rate.rename('EFETIVO', inplace=True)
df_rat = df_rat.join(df_rate, how='left')
del df_rate
df_rat['EFETIVO'].fillna(0, inplace=True)
df_rat['EFETIVO'] = df_rat['EFETIVO'].astype(int)

######################### BUSCA DADOS DE PRODUTIVIDADE E ACRESCENTA EM df_rat
df_ratp = pd.read_sql_table('tbl_rat_produtividade', 'sqlite:///gdo.db')
df_ratp_efet = df_ratp[
    df_ratp['DESCRICAO'].isin(vars.itens_efet)
].drop_duplicates('RAT.NUM_ATIVIDADE').set_index('RAT.NUM_ATIVIDADE')['QUANTIDADE'].map(lambda qtd:1).rename('EFETIVIDADE_PARCIAL')
df_rat = df_rat.join(df_ratp_efet, how='left')
del df_ratp_efet
df_rat['EFETIVIDADE_PARCIAL'].fillna(0, inplace=True)
df_rat['EFETIVIDADE_PARCIAL'] = df_rat['EFETIVIDADE_PARCIAL'].astype(int)

df_itens_ee = df_ratp[
    df_ratp['DESCRICAO'].isin([
        'Qde de veiculos fiscalizados',
        'Qde de pessoas abordadas',
        'Qde de locais fiscalizados',
        'Qde de pessoas que sopraram o etilometro'
    ])
]
df_itens_ee = pd.pivot_table(df_itens_ee, columns='DESCRICAO', index='RAT.NUM_ATIVIDADE')
df_itens_ee.columns = df_itens_ee.columns.droplevel()
df_rat = df_rat.join(df_itens_ee, how='left')
for i in [
    'Qde de veiculos fiscalizados',
    'Qde de pessoas abordadas',
    'Qde de locais fiscalizados',
    'Qde de pessoas que sopraram o etilometro'
]:
    df_rat[i].fillna(0, inplace=True)
    df_rat[i] = df_rat[i].astype('int32')
del df_ratp

cond71 = (
    # Y07001 - OPERACAO DE BATIDA POLICIAL
    (
        (df_rat['NAT.CODIGO'] == 'Y07001') &
        (df_rat['TEMPO_INT'] >= 30) &
        (df_rat['EFETIVO'] >= 2)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 5) |
        (df_rat['Qde de veiculos fiscalizados'] >= 2)
    )
)
cond73 = (
    # Y07003 - OPERACAO DE INCURSAO EM ZONA QUENTE DE CRIMINALIDADE
    (
        (df_rat['NAT.CODIGO'] == 'Y07003') &
        (df_rat['TEMPO_INT'] >= 30) &
        (df_rat['EFETIVO'] >= 3) &
        (df_rat['VIATURAS'] >= 1)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 5) |
        (df_rat['Qde de veiculos fiscalizados'] >= 2) |
        (df_rat['Qde de locais fiscalizados'] >= 2)
    )
)
cond74 = (
    # Y07004 - OPERACAO DE CERCO / BLOQUEIO / INTERCEPTACAO
    (
        (
            (df_rat['NAT.CODIGO'] == 'Y07004') &            
            (df_rat['EFETIVO'] >= 3) &
            (df_rat['VIATURAS'] >= 2)
        ) &
        (
            (df_rat['Qde de pessoas abordadas'] >= 1) |
            (df_rat['Qde de veiculos fiscalizados'] >= 1)
        )
    )
)
cond75 = (
    # Y07005 - OPERACAO DE OCUPACAO DE PTOS DE ZONA QUENTE DE CRIMINALIDADE
    (
        (df_rat['NAT.CODIGO'] == 'Y07005') &
        (df_rat['TEMPO_INT'] >= 60) &
        (df_rat['EFETIVO'] >= 3) &
        (df_rat['VIATURAS'] >= 1)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 5) |
        (df_rat['Qde de veiculos fiscalizados'] >= 2) |
        (df_rat['Qde de locais fiscalizados'] >= 2)
    )    
)
cond412 = (
    # Y04012 - OPERACAO LEI SECA
    (
        (df_rat['NAT.CODIGO'] == 'Y04012') &
        (df_rat['TEMPO_INT'] >= 30) &
        (df_rat['EFETIVO'] >= 2) &
        (df_rat['VIATURAS'] >= 1)
    ) &
    (
        (df_rat['Qde de pessoas abordadas'] >= 3) |
        (df_rat['Qde de pessoas que sopraram o etilometro'] >= 3) |
        (df_rat['Qde de veiculos fiscalizados'] >= 3)
    )
)

cond_ee = [
    cond71 |
    cond73 |
    cond74 |
    cond75 |
    cond412
]

df_rat['EFICIENCIA_E_EFICACIA'] = np.select(cond_ee,[1],default=0)
df_rat['EFETIVIDADE'] = np.select(
    [(df_rat['EFETIVIDADE_PARCIAL'] == 1) & (df_rat['EFICIENCIA_E_EFICACIA'] == 1)],
    [1],
    default=0
)

df_rat.head(3)

CPU times: user 2.54 s, sys: 30.9 ms, total: 2.57 s
Wall time: 2.57 s


,NAT.CODIGO,NAT.DESCRICAO,DTA_HRA_INCLUSAO,DTA_INICIO,HRA_INICIO,DTA_TERMINO,HRA_TERMINO,DES_ALVO_EVENTO,DES_LUGAR,NOM_OPERACAO,...,CIA,VIATURAS,EFETIVO,EFETIVIDADE_PARCIAL,Qde de locais fiscalizados,Qde de pessoas abordadas,Qde de pessoas que sopraram o etilometro,Qde de veiculos fiscalizados,EFICIENCIA_E_EFICACIA,EFETIVIDADE
RAT.NUM_ATIVIDADE,,,,,,,,,,,,,,,,,,,,,
2018-046472984-001,Y07001,OPERACAO DE BATIDA POLICIAL,17/10/2018,17/10/2018,18:43,17/10/2018,19:00,TRANSEUNTE,VIA DE ACESSO PUBLICA,,...,139 CIA,1,0,0,0,0,0,0,0,0
2018-051709149-001,Y07001,OPERACAO DE BATIDA POLICIAL,20/11/2018,20/11/2018,05:25,20/11/2018,07:15,TRANSEUNTE,METALURGICA / SIDERURGICA,,...,142 CIA,1,0,0,0,0,0,0,0,0
2018-052156427-001,Y07001,OPERACAO DE BATIDA POLICIAL,22/11/2018,22/11/2018,20:20,22/11/2018,20:45,TRANSEUNTE,VIA DE ACESSO PUBLICA,,...,142 CIA,1,0,0,0,0,0,0,0,0


In [69]:
df_rat[
    (df_rat['ANO'] == 2020) 
    &    (df_rat['MES'] == 4)
    & (~df_rat['NOM_UNID_RESPONSAVEL'].str.contains('PRV'))
    & (df_rat['NAT.CODIGO'] == 'Y04012')
].groupby(['ANO','MES','DEZENA','CIA','NAT.CODIGO']).sum()['EFETIVIDADE'].unstack(['ANO','MES','DEZENA'])

ANO                2020     
MES                   4     
DEZENA                1    2
CIA     NAT.CODIGO          
139 CIA Y04012      NaN  0.0
142 CIA Y04012      2.0  0.0
51 CIA  Y04012      0.0  NaN
53 CIA  Y04012      1.0  0.0

<h2 style='color:#009'>Código para gerar relatório de LEI SECA, por Cias, por ano, por mês</h2>

In [68]:
df_ols = df_rat[
    (df_rat.loc[:,'NAT.CODIGO'] == 'Y04012') &
    ( ~ df_rat.loc[:,'NOM_UNID_RESPONSAVEL'].str.contains('PRV')) &
    (df_rat.loc[:,'ANO'] == 2020)
]

df_ols.loc[:,'CIA_OLS'] = np.select([
    df_ols.loc[:,'NOM_UNID_RESPONSAVEL'].str.contains('240')
], [
    '240 CIA'
], default = df_ols.loc[:,'CIA'])

df_ols_resumo = df_ols.groupby(['ANO','MES','CIA_OLS']).sum()[['EFETIVIDADE']]


df_ols_resumo = df_ols_resumo.unstack(level='MES')
df_ols_resumo.fillna(0, inplace=True)

df_ols_resumo.loc[:,('EFETIVIDADE','ACUM')] = df_ols_resumo.sum(1)


df_ols_resumo.loc[(2020,'23 BPM'),:] = df_ols_resumo.sum()
df_ols_resumo.fillna(0, inplace=True)
df_ols_resumo = df_ols_resumo.astype('int32')

df_ols_resumo.rename(columns={'EFETIVIDADE':'OLS'}, inplace=True)
df_ols_resumo


OLS              
MES            1  2  3  4 ACUM
ANO  CIA_OLS                  
2020 139 CIA   1  0  1  0    2
     142 CIA   0  1  0  0    1
     240 CIA   0  1  2  3    6
     51 CIA    0  3  1  0    4
     53 CIA    2  0  0  0    2
     23 BPM    3  5  4  3   15

<h2 style='color:#009'>Código para gerar relatório de RQV, por setores, por ano, por mês</h2>

In [13]:
df_rqv_23 = df_rat[
    (df_rat['NAT.CODIGO'].str.contains('Y0700[1345]')) &
    ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('IND PE')) &
    ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('BPE')) &
    ( ~ df_rat['NOM_UNID_RESPONSAVEL'].str.contains('CPE')) &
    (df_rat['ANO'] == 2020)
            
].groupby(['ANO','MES','CIA']).sum()[['EFICIENCIA_E_EFICACIA','EFETIVIDADE']]


df_rqv_23 = df_rqv_23.unstack(level='MES')
df_rqv_23.columns = df_rqv_23.columns.set_levels(df_rqv_23.columns.levels[1].astype('int32'), level=1)
df_rqv_23.index = df_rqv_23.index.set_levels(df_rqv_23.index.levels[0].astype('int32'), level=0)
df_rqv_23.loc[:,('EFICIENCIA_E_EFICACIA','ACUM')] = df_rqv_23['EFICIENCIA_E_EFICACIA'].sum(1)
df_rqv_23.loc[:,('EFETIVIDADE','ACUM')] = df_rqv_23['EFETIVIDADE'].sum(1)
df_rqv_23 = df_rqv_23[['EFICIENCIA_E_EFICACIA','EFETIVIDADE']]
df_rqv_23.loc[(2020,'23 BPM'),:] = df_rqv_23.sum()
df_rqv_23.fillna(0, inplace=True)
df_rqv_23 = df_rqv_23.astype('int32')
df_rqv_23

EFICIENCIA_E_EFICACIA                      EFETIVIDADE            \
MES                              1    2    3    4  ACUM           1    2    3   
ANO  CIA                                                                        
2020 139 CIA                   214  234  208   98   754          44   39   37   
     142 CIA                   412  358  333  143  1246         142  141  138   
     51 CIA                    251  152  120   77   600          92   58   60   
     53 CIA                    221  173  187  101   682          54   30   27   
     other                       0    0    0    0     0           0    0    0   
     23 BPM                   1098  917  848  419  3282         332  268  262   

                        
MES             4 ACUM  
ANO  CIA                
2020 139 CIA   24  144  
     142 CIA   59  480  
     51 CIA    30  240  
     53 CIA    18  129  
     other      0    0  
     23 BPM   131  993

In [7]:
writer = pd.ExcelWriter('rqv_e_ols.xlsx', engine='xlsxwriter')
df_rqv_23.to_excel(writer, sheet_name='RQV')
df_ols_resumo.to_excel(writer, sheet_name='OLS')
writer.save()